# Running the Classifier



## Setting up the classes

This tutorial assumes the user has run the testing suite and has access to the testing data located in the `tests` directory. This can be done by running the following commands in the root directory of the repository.

In [ ]:
!git lfs fetch
!git lfs checkout

In [2]:
!pip install dcm-classifier==0.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 2.7 MB/s eta 0:00:00


In [3]:
from dcm_classifier.study_processing import ProcessOneDicomStudyToVolumesMappingBase
from dcm_classifier.image_type_inference import ImageTypeClassifierBase
from pathlib import Path


current_directory: Path = Path.cwd()
root_directory = current_directory.parent

session_directory = root_directory / "tests" / "testing_data" / "anonymized_testing_data" / "anonymized_data"

# create inferer object
inferer = ImageTypeClassifierBase()

# create study for given session directory
study = ProcessOneDicomStudyToVolumesMappingBase(
    study_directory=session_directory.as_posix(), inferer=inferer
)

# run the inference on the study
study.run_inference()

The directory: /home/cavriley/programs/dcm-classifier/tests/testing_data/anonymized_testing_data/anonymized_data contains 16 DICOM sub-volumes


## DICOM Selection

In many applications we are interested in only specific modalities or images acquired along specfic plane. dcm-classifier makes it easy to make this selection. 

The code below shows how to parse all images within the study and how to choose a specific image, in this example it: Axial T1w

NOTE: we can parse on a series or volume level. In most cases series has only one volume but in some cases such as Diffusion Weighted Imaging a series could have multiple volumes and have different `.get_volume_modality()` and `.get_series_modality()` outputs (Example: Volume 'b0' Series: 'dwig' meaning the B0 volume is one of the components of DWI image)

In [4]:
ax_t1w = None
for series_number, series in study.series_dictionary.items():
    for index, volume in enumerate(series.get_volume_list()):
        volume_modality = volume.get_volume_modality()
        volume_acq_plane = volume.get_acquisition_plane()
        print(f"Series Number: {series_number} | Modality: {volume_modality} | Acq Plane: {volume_acq_plane}")
        if volume_modality == "t1w" and volume_acq_plane == "ax":
            ax_t1w = volume.get_itk_image()

Series Number: 11 | Modality: t2w | Acq Plane: ax
Series Number: 13 | Modality: t1w | Acq Plane: sag
Series Number: 1 | Modality: gret2star | Acq Plane: sag
Series Number: 4 | Modality: gret2star | Acq Plane: ax
Series Number: 7 | Modality: flair | Acq Plane: ax
Series Number: 15 | Modality: t1w | Acq Plane: cor
Series Number: 5 | Modality: tracew | Acq Plane: ax
Series Number: 5 | Modality: tracew | Acq Plane: ax
Series Number: 6 | Modality: adc | Acq Plane: ax
Series Number: 9 | Modality: gret2star | Acq Plane: ax
Series Number: 3 | Modality: gret2star | Acq Plane: cor
Series Number: 2 | Modality: gret2star | Acq Plane: sag
Series Number: 10 | Modality: t1w | Acq Plane: sag
Series Number: 12 | Modality: t1w | Acq Plane: ax
Series Number: 14 | Modality: t1w | Acq Plane: ax
Series Number: 8 | Modality: gret2star | Acq Plane: ax


Our volume of interest is the T1w Axial image returned as ITK image

In [5]:
print(type(ax_t1w))
print(ax_t1w)

<class 'itk.itkImagePython.itkImageF3'>
Image (0x55fc3f6d0760)
  RTTI typeinfo:   itk::Image<float, 3u>
  Reference Count: 1
  Modified Time: 9228
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 4653
  UpdateMTime: 9227
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [192, 256, 21]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [192, 256, 21]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [192, 256, 21]
  Spacing: [0.9375, 0.9375, 7]
  Origin: [-91.4058, -112.463, -91.643]
  Direction: 
0.990214 0.056922 -0.127424
-0.061412 0.997613 -0.0315869
0.125322 0.0391031 0.991345

  IndexToPointMatrix: 
0.928325 0.0533643 -0.891968
-0.0575737 0.935262 -0.221108
0.117489 0.0366592 6.93942

  PointToIndexMatrix: 
1.05623 -0.0655061 0.133677
0.0607168 1.06412 0.04171
-0

## Saving Sequences

Saving volumetric data can be done in two ways here - using the ITK library or the popular [dcm2niix](https://github.com/rordenlab/dcm2niix) package. For these examples, we are saving the Axial T1W selected earlier. 

Note: the example data used has been anonymized and the pixel intensity information was set as 0 for all voxels

#### Using ITK

Once the image is obtained, the image can easily be saved using ITK. ITK is the preferred method for saving the images as it supports all image type supported by ITK such as NIfTI and NRRD in the example below.

In [ ]:
import itk

# let's follow the BIDS naming system
# Note: the information about the study or patient can be obtained using pydicom and a .dcm file corresponing to the volume
# However, here we will make up the subject and sesion ids
sub = "sub-01"
ses = "ses-01"
nifti_name = f"{sub}_{ses}_desc-ax_t1w.nii.gz"
nrrd_name = f"{sub}_{ses}_desc-ax_t1w.nrrd"

# save image as nifti and nrrd
itk.imwrite(ax_t1w, nifti_name)
itk.imwrite(ax_t1w, nrrd_name)

#### Handling multiple images of the same modality

We can have more than one image of the same modality. Here is a simple way how to save all the data utilizing the `run` parameter of BIDS format.

In [ ]:
sub = "sub-01"
ses = "ses-01"
output_dir = '.'
suffix = '.nii.gz'
desc = "usingITK"

for series_number, series in study.series_dictionary.items():
    for index, volume in enumerate(series.get_volume_list()):
        modality = volume.get_volume_modality()
        plane = volume.get_acquisition_plane()
        if modality == "t1w" and plane == "ax":
            itk_volume = volume.get_itk_image() # get image
            run_num = 1
            fname = f"{sub}_{ses}_run-00{run_num}_acq-{plane}_desc-{desc}_{modality}{suffix}" # set filename
            while Path(f"{output_dir}/{fname}").exists(): # ensure we increase run number instead of overwriting
                fname = fname.replace(f"run-00{run_num}", f"run-00{run_num+1}")
                run_num += 1
            itk.imwrite(itk_volume, fname)
            

#### Using dcm2niix

Using dcm2niix is more complicated. We provide a function that takes path to the dcm2niix script together with other information.

In [ ]:
from subprocess import run

def create_nifti_file(
    dcm2niix_path: str,
    subject: str,
    session: str,
    modality: str,
    plane: str,
    output_dir: str,
    dcm_dir: str,
    desc: str = None,
) -> None:
    run_num = 1
    if desc is not None:
        fname = (
            f"{subject}_{session}_run-00{run_num}_acq-{plane}_desc-{desc}_{modality}"
        )
    else:
        fname = f"{subject}_{session}_run-00{run_num}_acq-{plane}_{modality}"
    while Path(f"{output_dir}/{fname}.nii.gz").exists():
        fname = fname.replace(f"run-00{run_num}", f"run-00{run_num+1}")
        run_num += 1
    run(
        [dcm2niix_path, "-o", output_dir, "-f", fname, "-z", "y", dcm_dir]
    ) 
    print(f"{dcm2niix_path} -o {output_dir} -f {fname} -n -z y {dcm_dir}")

For robustness, the code below will move all individual DICOM files corresponging to the volume to a temporary directory. Then the dcm2niix script uses that directory to reed the files and save the output. The advantage of using the dcm2niix is that it produces the corresponding .json file the process is much more complicated and and slower comparing to ITK option above. 

In [ ]:
import tempfile
import shutil
    
dcm2niix_path = "PATH TO dcm2niix"

sub = "sub-01-test"
ses = "ses-01-test"
desc = "usingdcm2niix"

final_dir_path = "."

for series_number, series in study.series_dictionary.items():
    for index, volume in enumerate(series.get_volume_list()):
        if volume.get_volume_modality() == "t1w" and volume.get_acquisition_plane() == "ax":
            temp_dir = tempfile.mkdtemp()
            file_list = volume.get_one_volume_dcm_filenames()
            for file in file_list:
                shutil.copy(file, temp_dir)
            create_nifti_file(
                dcm2niix_path=dcm2niix_path,
                subject=sub,
                session=ses,
                modality=volume.get_volume_modality(),
                plane=volume.get_acquisition_plane(),
                output_dir=final_dir_path,
                dcm_dir=temp_dir,
                desc=desc
            )